# Compile the latex files for daily Tracks schedules for talks and posters

In [1]:
import collections
import os
from os import listdir, chdir,makedirs
import shutil
import io  
import sys  
import urllib
import csv
from pylatexenc.latexencode import utf8tolatex
from pylatexenc.latex2text import LatexNodes2Text

from time import strptime
from datetime import datetime
from datetime import date

from dumptruck import DumpTruck
from collections import OrderedDict
from collections import Counter
from collections import deque
from os import listdir, chdir,makedirs

# Functions for processing presentations into LaTeX formating for daily Schedules

In [2]:
def cleanLaTeX(textstring):
    a = utf8tolatex(textstring)
    a = a.replace("ɔ".decode('utf-8'), "\\textipa{O}")
    a = a.replace("ɛ".decode('utf-8'), "\\textipa{3}")
    a = a.replace("x̄".decode('utf-8'), "$\\bar{x}$")
    a = a.replace("↔".decode('utf-8'), "$\leftrightarrow$")
    a = a.replace("♭".decode('utf-8'), "$\\flat$")
    a = a.replace("| אוניברסיטת תל-אביב".decode('utf-8'), "")
    a = a.replace("ṭ".decode('utf-8'), "\d{t}").encode('utf-8')   
    #a = cleanLaTeX(textstring)
    return a

In [3]:
def talkDets_schedule(talk):
    TalkCode = ("%s %s-%i" % (talk['Time'],talk['SessionCode'],talk['TalkSessionNumber']))
    SS = ("\\textit{%s} & \\textbf{%s}\\\\\n" % (TalkCode,cleanLaTeX(talk['TalkTitle'])))

    authors = '& {\small '
    for i in range(len(talk['Authors'])):
        authors += cleanLaTeX(talk['Authors'][i].strip('*')) # Author
        if i < len(talk['Authors'])-1:
            authors += ', '
    authors += '} \\\\\n'
    texTalk = SS + authors
    
    return texTalk

In [4]:
def SympDets_schedule(talk):
    TalkCode = ("%s-%i" % (talk['SessionCode'],talk['TalkSessionNumber']))
    SS = ("\\textit{%s} & \\textbf{%s}\\\\\n" % (TalkCode,cleanLaTeX(talk['TalkTitle'])))
    authors = '& {\small '
    for i in range(len(talk['Authors'])):
        authors += cleanLaTeX(talk['Authors'][i].strip('*')) # Author
        if i < len(talk['Authors'])-1:
            authors += ', '
    authors += '} \\\\\n'
    texTalk = SS + authors
    
    return texTalk

In [11]:
def PosterDets_schedule(talk):
    TalkCode = ("%s-%i" % (talk['PosterSession'],talk['PositionNumber']))
    SS = ("\\textit{%s} & \\textbf{%s}\\\\\n" % (TalkCode,cleanLaTeX(talk['PosterTitle'])))
    authors = '& {\small '
    for i in range(len(talk['Authors'])):
        authors += cleanLaTeX(talk['Authors'][i].strip('*')) # Author
        if i < len(talk['Authors'])-1:
            authors += ', '
    authors += '} \\\\\n'
    texTalk = SS + authors
    
    return texTalk

# Run through presentation databases and produce pdfs of daily schedules in LaTex

In [9]:
chdir('./data')
dt = DumpTruck(dbname='Aug5_Talks.db')
ds = DumpTruck(dbname='Aug5_Sessions.db')
data = ds.dump()
chdir('..')
chdir('./Schedules')

# OK now write to file

fileName = "Aug5_Track"
Rooms = ['KC802','KC905/907','KC909','KC914']

for tracks in range(4):
    trackNumber = str(tracks + 1)

    texName =  "Sessions.tex"
    f= open(texName, 'w+')
    f.write('\\begin{center} \\textbf{\Huge August 5th, Track '+ trackNumber +', ' +Rooms[tracks]+'}\n\end{center}\n\n ')

    for i in range(len(data)/4):
        ss = ds.execute('SELECT * FROM `dumptruck` WHERE `SessionSlot` = ' + str(i+1) + ' AND `TrackNumber` = ' + trackNumber)
        #f.write('\\begin{table}[h]\n')
        f.write('\\begin{tabular}{ABC}\n')
        sessionTitle = ('\\textbf{\large %s} & \\textbf{\Large %s} & \\textbf{\Large %s}\\\\\n & & \\\\\n' % 
                        (ss[0]['Times'],ss[0]['SessionCode'], cleanLaTeX(ss[0]['SessionTitle'])))
        f.write(sessionTitle)
        f.write('\end{tabular}\n\n')
        sessionNumber = ss[0]['SessionNumber']
        ts = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(sessionNumber))    
        if ts[0]['Status'] == 'Accept Spoken': 
            f.write('\\begin{tabular}{DE}\n')
            for talk in ts:
                r = talkDets_schedule(talk)
                f.write(r)
        else:
            f.write('\\begin{tabular}{FG}\n')
            for talk in ts:
                r = SympDets_schedule(talk)
                f.write(r)
            #print r
        f.write('\end{tabular}\n')
        f.write('\n\\vspace{5mm}\n\n')
    f.write('\end{document} ') 

    f.close() 
    os.system("pdflatex Schedule_1.tex")
    os.rename('Schedule_1.pdf',  fileName + trackNumber + ".pdf" )
    
chdir('..')

In [10]:
chdir('./data')
dt = DumpTruck(dbname='Aug6_Talks.db')
ds = DumpTruck(dbname='Aug6_Sessions.db')
data = ds.dump()
chdir('..')
chdir('./Schedules')

# OK now write to file

fileName = "Aug6_Track"
Rooms = ['KC802','KC905/907','KC909','KC914']

for tracks in range(4):
    trackNumber = str(tracks + 1)

    texName =  "Sessions.tex"
    f= open(texName, 'w+')
    f.write('\\begin{center} \\textbf{\Huge August 6th, Track '+ trackNumber +', ' +Rooms[tracks]+'}\n\end{center}\n\n ')

    for i in range(len(data)/4):
        ss = ds.execute('SELECT * FROM `dumptruck` WHERE `SessionSlot` = ' + str(i+1) + ' AND `TrackNumber` = ' + trackNumber)
        #f.write('\\begin{table}[h]\n')
        f.write('\\begin{tabular}{ABC}\n')
        sessionTitle = ('\\textbf{\large %s} & \\textbf{\Large %s} & \\textbf{\Large %s}\\\\\n & & \\\\\n' % 
                        (ss[0]['Times'],ss[0]['SessionCode'], cleanLaTeX(ss[0]['SessionTitle'])))
        f.write(sessionTitle)
        f.write('\end{tabular}\n\n')
        sessionNumber = ss[0]['SessionNumber']
        ts = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(sessionNumber))    
        if ts[0]['Status'] == 'Accept Spoken': 
            f.write('\\begin{tabular}{DE}\n')
            for talk in ts:
                r = talkDets_schedule(talk)
                f.write(r)
        else:
            f.write('\\begin{tabular}{FG}\n')
            for talk in ts:
                r = SympDets_schedule(talk)
                f.write(r)
            #print r
        f.write('\end{tabular}\n')
        f.write('\n\\vspace{5mm}\n\n')
    f.write('\end{document} ') 

    f.close() 
    os.system("pdflatex Schedule_1.tex")
    os.rename('Schedule_1.pdf',  fileName + trackNumber + ".pdf" )
chdir('..')

In [8]:
chdir('./data')
dt = DumpTruck(dbname='Aug7_Talks.db')
ds = DumpTruck(dbname='Aug7_Sessions.db')
data = ds.dump()
chdir('..')
chdir('./Schedules')

# OK now write to file

fileName = "Aug7_Track"
Rooms = ['KC802','KC905/907','KC909','KC914']

for tracks in range(4):
    trackNumber = str(tracks + 1)

    texName =  "Sessions.tex"
    f= open(texName, 'w+')
    f.write('\\begin{center} \\textbf{\Huge August 7th, Track '+ trackNumber +', ' +Rooms[tracks]+'}\n\end{center}\n\n ')

    for i in range(len(data)/4):
        ss = ds.execute('SELECT * FROM `dumptruck` WHERE `SessionSlot` = ' + str(i+1) + ' AND `TrackNumber` = ' + trackNumber)
        #f.write('\\begin{table}[h]\n')
        f.write('\\begin{tabular}{ABC}\n')
        sessionTitle = ('\\textbf{\large %s} & \\textbf{\Large %s} & \\textbf{\Large %s}\\\\\n & & \\\\\n' % 
                        (ss[0]['Times'],ss[0]['SessionCode'], cleanLaTeX(ss[0]['SessionTitle'])))
        f.write(sessionTitle)
        f.write('\end{tabular}\n\n')
        sessionNumber = ss[0]['SessionNumber']
        ts = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(sessionNumber))    
        if ts[0]['Status'] == 'Accept Spoken': 
            f.write('\\begin{tabular}{DE}\n')
            for talk in ts:
                r = talkDets_schedule(talk)
                f.write(r)
        else:
            f.write('\\begin{tabular}{FG}\n')
            for talk in ts:
                r = SympDets_schedule(talk)
                f.write(r)
            #print r
        f.write('\end{tabular}\n')
        f.write('\n\\vspace{5mm}\n\n')
    f.write('\end{document} ') 

    f.close() 
    os.system("pdflatex Schedule_1.tex")
    os.rename('Schedule_1.pdf',  fileName + trackNumber + ".pdf" )
chdir('..')

In [12]:
# Posters
chdir('./data')
dt = DumpTruck(dbname='Aug6_Posters.db')
data = ds.dump()
chdir('..')
chdir('./Schedules')

# OK now write to file

fileName = "Aug6_Posters"
sessions = ['1','2']


for sess in sessions:
    reOrder = []
    texName =  "List.tex"
    f= open(texName, 'w+')
    ps = dt.execute('SELECT * FROM `dumptruck` WHERE `PosterSessionNumber` = ' + sess)
    sessionTitle = ('\\begin{center} \\textbf{\Huge  %s Poster Session %s, Order List}\n\end{center}\n' %  (ps[0]['Time'], ps[0]['PosterSession']))
    f.write(sessionTitle )
    f.write('\\begin{tabular}{FG}\n')
    
    for poster in ps:
        r = PosterDets_schedule(poster)
        reOrder.append([poster['AuthorIndex'][0],poster])
        f.write(r)
    f.write('\n\end{tabular}\n')
    f.write('\end{document} ') 

    f.close() 
    os.system("pdflatex Posters_1.tex")
    os.rename('Posters_1.pdf',  fileName + sess + ".pdf" )
    
    reOrder.sort()
    
    texName =  "List.tex"
    f= open(texName, 'w+')
    ps = dt.execute('SELECT * FROM `dumptruck` WHERE `PosterSessionNumber` = ' + sess)
    sessionTitle = ('\\begin{center} \\textbf{\Huge  %s Poster Session %s, Alphabetical List}\n\end{center}\n' %  (ps[0]['Time'], ps[0]['PosterSession']))
    f.write(sessionTitle )
    f.write('\\begin{tabular}{FG}\n')

    for entry in reOrder:
        r = PosterDets_schedule(entry[1])
        f.write(r)
    f.write('\n\end{tabular}\n')
    f.write('\end{document} ') 

    f.close() 
    os.system("pdflatex Posters_1.tex")
    os.rename('Posters_1.pdf',  fileName + sess + "_alphabetical.pdf" )
chdir('..')

In [13]:
# Posters
chdir('./data')
dt = DumpTruck(dbname='Aug7_Posters.db')
data = ds.dump()
chdir('..')
chdir('./Schedules')

# OK now write to file

fileName = "Aug7_Posters"
sessions = ['3','4']

for sess in sessions:
    reOrder = []
    texName =  "List.tex"
    f= open(texName, 'w+')
    ps = dt.execute('SELECT * FROM `dumptruck` WHERE `PosterSessionNumber` = ' + sess)
    sessionTitle = ('\\begin{center} \\textbf{\Huge  %s Poster Session %s, Order List}\n\end{center}\n' %  (ps[0]['Time'], ps[0]['PosterSession']))
    f.write(sessionTitle )
    f.write('\\begin{tabular}{FG}\n')
    
    for poster in ps:
        r = PosterDets_schedule(poster)
        reOrder.append([poster['AuthorIndex'][0],poster])
        f.write(r)
    f.write('\n\end{tabular}\n')
    f.write('\end{document} ') 

    f.close() 
    os.system("pdflatex Posters_1.tex")
    os.rename('Posters_1.pdf',  fileName + sess + ".pdf" )
    
    reOrder.sort()
    
    texName =  "List.tex"
    f= open(texName, 'w+')
    ps = dt.execute('SELECT * FROM `dumptruck` WHERE `PosterSessionNumber` = ' + sess)
    sessionTitle = ('\\begin{center} \\textbf{\Huge  %s Poster Session %s, Alphabetical List}\n\end{center}\n' %  (ps[0]['Time'], ps[0]['PosterSession']))
    f.write(sessionTitle )
    f.write('\\begin{tabular}{FG}\n')

    for entry in reOrder:
        r = PosterDets_schedule(entry[1])
        f.write(r)
    f.write('\n\end{tabular}\n')
    f.write('\end{document} ') 

    f.close() 
    os.system("pdflatex Posters_1.tex")
    os.rename('Posters_1.pdf',  fileName + sess + "_alphabetical.pdf" )
chdir('..')